<a href="https://colab.research.google.com/github/GimenesPaula/GimenesPaula/blob/main/Projeto%201.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fazer Upload planilhas
1.   Launches
2.   Inci
3.   Sales
4.   Project

In [1]:
#Launches
from google.colab import files
uploaded = files.upload()

Saving 2024_16_08.xlsx to 2024_16_08.xlsx


In [2]:
filename = next(iter(uploaded))

In [3]:
# INCI name produtos
from google.colab import files
inci = files.upload()

Saving inci_16_08.xlsx to inci_16_08.xlsx


In [4]:
filename2 = next(iter(inci))

In [5]:
#Projetos Distribuidores
from google.colab import files
proj = files.upload()

Saving Projetos_20_08.xlsx to Projetos_20_08.xlsx


In [6]:
filename3 = next(iter(proj))

In [7]:
#Vendas Distribuidores
from google.colab import files
dist = files.upload()

Saving Vendas_16_08.xlsx to Vendas_16_08.xlsx


In [8]:
filename4 = next(iter(dist))

# Bibliotecas Phyton

In [9]:
#Carrega Bibliotecas
import pandas as pd
import numpy as np
import re

In [10]:
pip install unidecode

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 3.5 MB/s eta 0:00:00


In [11]:
from unidecode import unidecode

#Carrega Bases de Dados 1) Lançamento; 2) INCI; 3) Projetos; 4) Vendas

## Upload Lançamentos





In [65]:
#Carrega o banco de dados como tabela
data = '/content/'+filename
df_launches = pd.read_excel(data)
pd.options.display.max_columns = None
df_launches.nunique()

,0
Record ID,2055
Date/Published,123
Product,1358
Brand,1372
Company,529
Market,1
Category,4
Sub-Category,38
Product Description,2054
Launch Type,5


##Upload INCI

In [66]:
data = '/content/'+filename2
df_inci = pd.read_excel(data)
pd.options.display.max_columns=None
df_inci.nunique()

,0
Produto,50
Ingrediente,49
Prioridade,3


##Upload Projetos

In [67]:
data = '/content/'+filename3
df_proj = pd.read_excel(data)
pd.options.display.max_columns=None
df_proj.nunique()

,0
Distributor,4
Customer,812
UF,19
Material,183
Inicio,1297
Status,17


## Upload Vendas

In [68]:
data = '/content/'+filename4
df_sale = pd.read_excel(data)
pd.options.display.max_columns=None
df_sale.nunique()

,0
Customer(at distrib),811
Year,1
Distributor,4
Material,156
Volume,273


# Define funções

## Procura INCI

In [69]:
#this checks if any combination of INCI as present in Ingredient
def verifica_ingrediente(formula,produto,material):
  quantidade = len(produto.difference(formula))
  if quantidade == 0:
    return material
  return None

In [70]:
#If last code is true, this returns the material name
def procura_produtos(formula, produtos, materiais):
  formula = formula.copy()
  resultados = []
  for prod, mat in zip(produtos, materiais):
    resultado = verifica_ingrediente(formula, prod, mat)
    if resultado is not None:
      formula = formula.difference(prod) ## Para remover os ingredientes já encontrados numa nova busca.
      resultados.append(resultado)
  return resultados

## Transpoe coluna

In [71]:
#this code transpose data. Used when we bring each category and the number of lauches.
def transpor (df, coluna, linha):
  for cat in df[coluna].unique():
    f = df[coluna] == cat
    df[cat] = df[f][linha]
    f = df[cat].isna()
    df.loc[f, cat] = df.loc[f, cat].apply(lambda x:[])

In [72]:
def to_set(x):
  try:
    if isinstance(x, pd.DataFrame):
        return x.apply(to_set)
    else:
        return set(x.sum())
  except:
        pass

In [73]:
def transpor_2 (df, coluna, linha):
  for cat in df[coluna].unique():
    f = df[coluna] == cat
    df[cat] = df[f][linha]
    f = df[cat].isna()
    df.loc[f, cat] = df.loc[f, cat].apply(lambda x:x)

## Remove palavras do dicionário, visando reduzir campo a indice comum

###Fabricante

In [74]:
#Usado para Fabricantes / Distribuidor
def formatar_fabricantes(coluna):
  remover = [
             'de castro', 'do brasil', 'brasil',
             'fabrica', 'fabricacao', 'manuf', 'tercerizacao', 'manufacturing',
             'farmacia', 'farmacias', 'farmaceuticos', 'drogaria','drogarias', 'farm',
             'cosmetico', 'cometico','cosmeticos', 'cometicos', 'cosmetics', 'cosmetic',
             'cosmiatria', 'cosmetica', 'cosmeticas', 'cosm', 'cosme', 'kosmetics', 'cosmet', 'cosmesticos',
             'ltda','epp', 'eireli', 'me', 'eirell', 'sa', 'eir', 'ltd',
             'indústria','industria', 'industrias', 'industrial', 'indutrial', 'ind', 'industriae',
             'comercio', 'comecio', 'comercial', 'corporation', 'com', 'de ae',
             'imp', 'exp',  'prod', 'exportacao','importacao','importadora', 'import',
             'do', 'e', 'para', 'de', 'le', 'la',
             'aerossois', 'aerosol', 'aerossol', 'tecnologia', 'profissional',
             'laboratorio','laboratorios', 'laboratories', 'laboratoire', 'instituto', 'group',
             'pesquisa', 'desenvolvimento', 'naturais', 'professional', 'aromatizantes',
             'produtos', 'Produto', 'saneantes', 'quimica', 'quimicos', 'quimico', 'farmaceutica', 'veterinário',
             'higiene','pessoal', 'limpeza', 'botanico', 'beauty',
             'perfumes','fragrancias', 'perfumaria', 'medicamentos', 'perf',
             'distribuidora', 'dist', 'amostras', 'faturamento',
             'international', 'internacional', 'sulamericana', 'nacional',
             'nbsp;', 'veer','serv', 'vajerista', 'amaral', 'jbs', 'nova',  'bioage',
           ]
  pat = '\\b({})\\b'.format('|'.join(remover))
  coluna = (
    coluna
    .apply(unidecode)
    .str.lower()
    .replace("\(.*?\)",'',regex=True)
    .replace('\.', '',regex=True)
    .replace('\/', '',regex=True)
    .replace('\-', '',regex=True)
    .replace(pat,'',regex=True)
    .replace(',', '',regex=True)
    .replace('\s+','',regex=True) #Tirar numeros
    .str.strip()
  )
  return coluna

###Material

In [75]:
#Dis dict clean the name of material
def formatar_material(coluna):
  remover = ['BELSIL®', 'BELSIL', 'BELSIM',
             'DRUM', 'ST', 'PE', 'CANISTER', 'RD', 'CARTON', 'IBC PE', '2', 'WOPAL',
             'SBPAL', 'STPAL', 'HOBBOCK', 'SACK', 'BOTTLE',
             '0,45KG','10KG','15KG','25KG', '30G', '50G', '60G', '100KG', '140KG', '150KG', '160KG',
             '180KG', '190KG', '200KG', '210KG', '900KG', '950KG', '190', '20KG',
             'KG', 'AMOSTRA', 'PA', 'XY', 'CP', 'SILICONE ELASTOMER GEL',
             'BR', 'DE', ' E', 'BLOCKED',
             'EMB. 0,100 KG', 'Ver  Pf 100', 'USP',
           ]
  pat = '\\b({})\\b'.format('|'.join(remover))
  coluna = (
    coluna
    .replace(pat,'',regex=True)
    .replace('\#','', regex=True)
    .replace('\"','', regex=True)
    .str.replace('\/', '',regex=True)
    .str.replace('\-', '',regex=True)
    .str.replace('\*', '',regex=True)
    .str.title()
    .str.strip()
  )
  return coluna

# Dicionários

## Status

In [76]:
dict_status = {'aprovado':'Won', 'closed won': 'Won',
             'cancelado': 'Closed', 'concluido':'Closed', 'stand by':'Closed', 'finalizado':'Closed',
             'closed lost': 'Lost', 'cancelado/reprovado':'Lost', 'reprovado':'Lost',
             'em andamento': 'Active', 'teste':'Active', 'em teste':'Active',
             'prospecting':'Active', 'qualification':'Active',
           }

## Estados

In [77]:
dict = {'AC':'AC', 'AL':'AL', 'AM':'AM', 'AP':'AP', 'BA':'BA', 'CE':'CE', 'DF':'DF', 'ES':'ES', 'GO':'GO', 'MA':'MA',
     'MG':'MG', 'MS':'MS', 'MT':'MT', 'PA':'PA', 'PB':'PB', 'PE':'PE', 'PI':'PI', 'PR':'PR', 'RJ':'RJ', 'RN':'RN',
     'RO':'RO', 'RR':'RR', 'RS':'RS', 'SC':'SC', 'SE':'SE', 'SP':'SP', 'TO':'TO', 'Sao Paulo':'SP', 'São Paulo':'SP',
     'Sao Paolo':'SP','Osasco': 'SP', 'Mogi das Cruzes': 'SP', 'Cotia':'SP', 'Diadema':'SP', 'São Roque':'SP',
     'São Sebastião':'SP', 'Lins':'SP', 'Cajamar':'SP', 'Pirassununga':'SP', 'Paulista': 'SP', 'São Paolo':'SP',
     'Elisario':'SP', 'Valinhos':'SP', 'Piracaia':'SP', 'São José do Rio Preto':'SP', 'Sao José do Rio Preto':'SP',
     'São José dos Campos':'SP',      'Araras':'SP', 'São Bernardo do Campo':'SP', 'São José dos Campos':'SP',
     'Sao Caetano do Sul':'SP', 'São José do Rio Preto':'SP', 'Brodowski':'SP', 'Cajuru':'SP', 'Itaquaquecetuba':'SP',
     'Itupeva':'SP', 'SO':'SP', 'Mairiporã':'SP', 'Itaquera':'SP', 'Franco da Rocha':'SP', 'Guarulhos':'SP',
     'Bairro Jardim America':'SP', 'Jardim Brazil':'SP', 'Agudos':'SP', 'Itapira':'SP', 'Rib. Preto':'SP', 'Vinhedo':'SP',
     'Lorena':'SP', 'Caieiras': 'SP', 'Louveira':'SP', 'Taboao da Serra':'SP', 'Hortôlandia':'SP', 'Americana':'SP',
     'Macatuba':'SP', 'Boituva':'SP', 'Jandira':'SP', 'Bom Jesus':'SP', 'Atibaia':'SP',
     'Ribeirão Preto':'SP', 'Cabreúva': 'SP', 'Presidente Prudente':'SP', 'Jurubatuba':'SP',
     'Jaú':'SP',
     'Rio de Janeiro':'RJ', 'Rio De Janeiro':'RJ', 'Nova Iguaçu': 'RJ', 'Nova Iguacu': 'RJ', 'Nivopolis':'RJ',
     'Duque de Caixas':'RJ', 'Engenheiro Pedreira':'RJ', 'Areal':'RJ', 'Duque de Caxias':'RJ',
     'Vila Velha':'ES', 'Marechal Floriano':'ES', 'Cariacica':'ES', 'Bom Pastor':'ES',
     'Belo Horizonte':'MG', 'Minas Gerais':'MG', 'Pocos de Caldas':'MG', 'Viçosa':'MG', 'Vicosa':'MG',
     'Pouso Alegre': 'MG', 'Uberlândia':'MG', 'Uberlandia':'MG', 'Ipatinga':'MG', 'Mirabela':'MG','Nova Lima':'MG',
     'São José da Lapa':'MG', 'Sao José da Lapa':'MG', 'Sao Jose da Lapa':'MG', 'Divinópolis':'MG',
     'Tres Pontas':'MG', 'Castelo BH':'MG', 'Varginha':'MG', 'Elói Mendes':'MG', 'Sete Lagoas':'MG',
     'Lagoa Santa':'MG',
     'Porto Alegre':'RS', 'Estância Velha': 'RS', 'Caxias do Sul':'RS', 'Passo Fundo':'RS', 'Gravatai':'RS',
     'Panambi':'RS', 'Gramado':'RS', 'Roca Sales':'RS',
     'Santa Catarina':'SC', 'Joinville':'SC','Camboriu':'SC', 'Maluche':'SC', 'Araquari':'SC', 'Criciúma': 'SC',
     'Criciuma': 'SC', 'Chapecó':'SC', 'Três Barras': 'SC', 'Palhoça':'SC', 'Palhoca':'SC',
     'Colombo':'PR', 'Paraná':'PR', 'Curitiba':'PR', 'Sao José dos Pinhais':'PR','BR 116 Km 85':'PR',
     'Cambé':'PR', 'Fazenda Rio Grande':'PR', 'Mandaguaçu':'PR', 'Pinhais':'PR',
     'Goiana':'GO', 'Goiânia':'GO', 'Anápolis':'GO', 'Anapolis':'GO', 'Go':'GO', 'Itapuranga': 'GO',
     'Guapó':'GO', 'Itajaí':'GO',
     'Brasília': 'DF', 'Brasilia': 'DF',
     'Rondônia':'RO','Novo Hamburgo':'RO','Manaus':'AM', 'Pará':'PA',
     'Bahia':'BA', 'Lauro de Freites': 'BA', 'Lauro de Freitas': 'BA', 'Av. Queira Deus':'BA',
     'Vila Cia Sul':'BA', 'Juazeiro':'BA',
     'Recife':'PE', 'Camaragibe': 'PE', 'Abreu e Lima':'PE', 'Fortaleza':'CE', 'Ceará':'CE', 'Caucaia':'CE',
     'Aracaju': 'SE', 'Nossa Sra. do Socorro':'SE',
     }

## Clientes

In [78]:
dictCustomer  = {'haskell':'gram', 'amendbemestar':'bemestar', 'belezanatural':'cor', 'cedralinoar':'cedral', 'novacedral(dvr)inoar':'cedral'}

# Relatório de Vendas

In [79]:
# remove textos desnecessários em Material
df_sale['Material']= formatar_material(df_sale['Material'])
df_sale['Material'] = df_sale['Material'].astype(str)

In [80]:
#Cria um indice para a coluna Fabricante
df_sale['KeyManuf'] = formatar_fabricantes(df_sale['Customer(at distrib)'])
df_sale['Distributor'] = formatar_fabricantes(df_sale['Distributor'])

In [81]:
for key in dictCustomer.keys():
    df_sale['KeyManuf'] = df_sale['KeyManuf'].replace(key, dictCustomer[key])

In [82]:
#agrupo os produtos em uma lista, por cliente
df_sale_unic = df_sale.groupby(['KeyManuf','Distributor']).agg({'Material':lambda x:list(set(x))}).reset_index()

In [83]:
#tranforma lista acima em objeto iterável, e renomeia coluna
df_sale_unic['Material'] = df_sale_unic['Material'].apply(set)
df_sale_unic.rename(columns={'Material': 'Sales'}, inplace=True)

In [84]:
#remove duplicadas
df_sale_unic.drop_duplicates(subset=['KeyManuf', 'Distributor'], keep='first', inplace=True)

# Relatório Projetos

In [85]:
# Reduz as colunas Fabricantes, Material e Distribuidor, a indice
df_proj['Material']= formatar_material(df_proj['Material'])
df_proj['Distributor'] = formatar_fabricantes(df_proj['Distributor'])
df_proj['KeyManuf'] = formatar_fabricantes(df_proj['Customer'])
df_proj['Customer'] = df_proj['Customer'].astype(str)
df_proj['Status'] = df_proj['Status'].astype(str)

In [86]:
df_proj['Status'] = df_proj['Status'].str.lower().map(dict_status)
#Exibe a quantidade de casos únicos
df_proj['Status'].value_counts(dropna=False)

,count
Status,
Active,2664
Won,512
Lost,87
Closed,42


In [87]:
for key in dictCustomer.keys():
    df_proj['KeyManuf'] = df_proj['KeyManuf'].replace(key, dictCustomer[key])

In [88]:
#agrupo todos os produtos em projeto em uma lista, por cliente
df_proj_a = df_proj.groupby(['KeyManuf','Distributor','Status']).agg({'Material':lambda x:list(set(x))}).reset_index()

In [89]:
transpor(df_proj_a, 'Status', 'Material')

In [90]:
df_proj_unic = df_proj_a.groupby(['KeyManuf', 'Distributor'])[['Active', 'Won', 'Closed', 'Lost']].apply(to_set)

# Agrupar Vendas Projetos

In [91]:
#Baseado no relatório de Projetos, verifica se ocorreram vendas
df_proj_vend = df_proj_unic.merge(df_sale_unic, on=['KeyManuf', 'Distributor'], how='left')

In [92]:
#Baseado no relatório de vendas, verifica-se se ocorreram projetos.
df_vend_proj = df_sale_unic.merge(df_proj_unic, on=['KeyManuf', 'Distributor'], how='left')

In [93]:
#Agrupa relatórios acima. O primeiro incluirá clientes com projetos, mas sem vendas. O segundo incluirá clientes com vendas, mas sem projetos.
df_vend_proj_cons = pd.concat([df_proj_vend, df_vend_proj])
df_vend_proj_cons.drop_duplicates(subset=['KeyManuf', 'Distributor'], keep='first', inplace=True)
df_vend_proj_cons.to_excel('Vendas e Projetos.xlsx')

# Mintel

## Define Estados


In [94]:
df_launches['State'] = df_launches['Manufacturer Company Address'].str.extract('('+'|'.join(dict)+')',expand=False).map(dict)
#Exibe a quantidade de casos únicos
df_launches['State'].value_counts(dropna=False)
#Em output se filtra estados vazios

,count
State,
SP,960
NaN,458
MG,127
RJ,116
PR,83
GO,81
RS,71
SC,49
CE,22


## Contém Silicone?

In [95]:
#Dicionário Silicones geral
dictOTHERS = {'methicone':'1', 'Dimethicone':'1','methicone Crosspolymer':'1','methiconol':'1',
              'ylsiloxysilicate':'1', 'ylsilsesquioxane':'1', 'Disiloxane':'1', 'Silica':'1','siloxane':'1'}

In [96]:
df_launches['Silicone'] = df_launches['Ingredients (Standard form)'].str.extract('('+'|'.join(dictOTHERS)+')',expand=False).map(dictOTHERS)

## Claim Livre de Silicones?


In [97]:
dictSil = {'Silicone':'1'}

In [98]:
df_launches['Free from'] = df_launches['Free from'].str.extract('('+'|'.join(dictSil)+')',expand=False).map(dictSil)

## Busca Produtos com INCI listado



In [99]:
df_inci.dropna(inplace=True)
df_inci['Produto'] = formatar_material(df_inci['Produto'])

In [100]:
#Cria uma lista iterável dos ingredientes nos Produtos
df_inci['Ing'] = formatar_material(df_inci['Ingrediente']).str.split(', ').apply(set)

In [101]:
#Conta a quantidade de ingredientes
df_inci.sort_values('Prioridade', ascending=True, inplace=True)

In [102]:
 #Dicionário de palavras a remover da coluna Ingredientes no Mintel
 dictIng={' Andor ':',',
          ', ':',',
          ' ,':',',
          ' (And) ':',',
          }

In [103]:
#Cria uma lista iterável das formulas do mintel
df_launches['Ingredients (Standard form)'] = df_launches['Ingredients (Standard form)'].astype(str)
df_launches['Ing'] = formatar_material(df_launches['Ingredients (Standard form)']).replace(dictIng,regex=True).dropna().str.split(',').apply(set)

In [104]:
#Aplica a função Procura Produtos
df_launches[''] = df_launches['Ing'].apply(lambda formulacao:procura_produtos(formulacao, df_inci['Ing'], df_inci['Produto']))

## Remover Clientes Diretos


In [105]:
dictClientes = {'Devintex Cosméticos':'1', 'Devintex':'1','Procosa Produtos de Beleza':'1', 'Colgate-Palmolive':'1', 'Unilever':'1',
                'Savoy Indústria de Cosméticos':'1', 'Beiersdorf':'1', 'Johnson & Johnson':'1',
                'Procter & Gamble':'1','Química Amparo':'1', "Oréal":'1', 'Procosa':'1',
                'Unilever Brasil Industrial':'1', 'Botica Comercial Farmacêutica':'1'}
dictMarcas = { 'Boticário':'1', 'Siàge':'1', 'Natura ':'1', 'Avon ':'1', 'Eudora':'1',
              'Wella':'1', 'The Body Shop':'1', 'Quem Disse,':'1', 'Salon Line':'1', 'Maybelline':'1'}
df_launches['Clientes']= df_launches['Manufacturer'].str.extract('('+'|'.join(dictClientes)+')',expand=False).map(dictClientes)
df_launches['Clientes2'] = df_launches['Brand'].str.extract('('+'|'.join(dictMarcas)+')',expand=False).map(dictMarcas)
df_ = df_launches[df_launches['Clientes']=='1']
df_ = df_[df_['Clientes2']=='1']
df_.to_excel("LancClientes.xlsx")

In [106]:
#Remove clientes  da base de tratamento
df_launches = df_launches[df_launches['Clientes']!='1']
df_launches = df_launches[df_launches['Clientes2']!='1']

## Relatório Linha a Linha



In [107]:
#Cria a coluna que irá indexar a mistura com a planilha do mintel
df_launches['Manufacturer']=df_launches['Manufacturer'].astype(str)
df_launches['KeyManuf'] = formatar_fabricantes(df_launches['Manufacturer'])

In [108]:
col=['Sub-Category', 'Product', 'Brand', 'Company', 'Manufacturer', 'KeyManuf', 'State', 'Free from', 'Price in local currency',
     'Silicone', '', 'Ing', 'Manufacturer Company Address',]

In [109]:
df_launches[col].to_excel("Lancamentos Linha a Linha.xlsx")

## Gera Relatório de Lançamentos com Categorias Transposta

In [110]:
df_launches['indice'] = 1

In [111]:
transpor_2(df_launches, 'Category', 'indice')

In [112]:
#renomeia coluna categoria
df_launches.rename(columns={'Skincare':'Skin', 'Hair Products':'Hair',
                          'Soap & Bath Products':'Bath', 'Colour Cosmetics': 'Make',
                            'Record ID':'Total Lançamentos'},inplace=True)

In [113]:
df_c = df_launches.groupby(['KeyManuf', 'State'])[['Skin', 'Hair', 'Make', 'Bath']].apply(lambda x:x.count())

In [114]:
#cria tabela que lista o fabricante, o estado e o total de lançamentos, quais tem silicone,
#e quais tem o claim sem silicone.
df_b = df_launches.groupby(['KeyManuf', 'State']).agg({
                                            'Total Lançamentos': 'count',
                                             'Silicone':'count',
                                             'Free from':'count',
                                              '':to_set})

In [115]:
#Une tabelas anteriores
df_launches_fab = pd.concat([df_c, df_b], axis=1).reset_index()

# Group all reports

In [116]:
#agrupa lançamentos com vendas e projetos
df_launches_vend_proj = df_launches_fab.merge(df_vend_proj_cons, on='KeyManuf', how='left')

# Generate a Report will all information

In [117]:
df_launches_vend_proj.to_excel('Final.xlsx')